In [ ]:
'''program 5
Implement a deep learning model using stacked RBMs.
Train layer-wise and fine-tune using backpropagation.
Compare performance with traditional deep networks.'''


#Train Layer-Wise Using RBMs
import numpy as np
from sklearn.neural_network import BernoulliRBM
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

# Load dataset (example: MNIST-like dataset - Digits dataset)
digits = load_digits()
X = digits.data
y = digits.target

# Preprocess data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Step 1: Train RBMs layer-wise (unsupervised pretraining)
def train_rbm_layer(X_train, n_components=64):
    rbm = BernoulliRBM(n_components=n_components, learning_rate=0.01, n_iter=10, random_state=42)
    rbm.fit(X_train)
    return rbm

# Train first RBM layer
rbm1 = train_rbm_layer(X_train, n_components=128)

# Transform the input data using the trained RBM
X_rbm1_train = rbm1.transform(X_train)
X_rbm1_test = rbm1.transform(X_test)

# Train second RBM layer
rbm2 = train_rbm_layer(X_rbm1_train, n_components=64)
X_rbm2_train = rbm2.transform(X_rbm1_train)
X_rbm2_test = rbm2.transform(X_rbm1_test)


In [ ]:
# Step 2: Fine-tune using Backpropagation
# Now we'll use the output of the last RBM (X_rbm2_train) as the input to a DNN classifier.

model = Sequential()
model.add(Dense(128, input_dim=X_rbm2_train.shape[1], activation='relu'))  # Layer 1
model.add(Dense(64, activation='relu'))  # Layer 2
model.add(Dense(10, activation='softmax'))  # Output layer (10 classes for digits dataset)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model (fine-tuning with labeled data)
model.fit(X_rbm2_train, y_train, epochs=10, batch_size=32, validation_data=(X_rbm2_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_rbm2_test, y_test)
print(f"Test accuracy (with RBM pretraining + backprop): {accuracy * 100:.2f}%")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.2637 - loss: 2.0900 - val_accuracy: 0.5250 - val_loss: 1.4649
Epoch 2/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5408 - loss: 1.3561 - val_accuracy: 0.5500 - val_loss: 1.2816
Epoch 3/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5619 - loss: 1.2082 - val_accuracy: 0.5778 - val_loss: 1.1946
Epoch 4/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.5518 - loss: 1.1997 - val_accuracy: 0.5972 - val_loss: 1.1610
Epoch 5/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6196 - loss: 1.0971 - val_accuracy: 0.5944 - val_loss: 1.1494
Epoch 6/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.5868 - loss: 1.1342 - val_accuracy: 0.6306 - val_loss: 1.1331
Epoch 7/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.6235 - loss: 1.0596 - val_accuracy: 0.6083 - val_loss: 1.1101
Epoch 8/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6333 - loss: 1.0446 - val_accuracy: 0.6417 - val_

In [ ]:


# Step 3: Train a Traditional Deep Neural Network (DNN) for Comparison

# Build a simple deep neural network (DNN)
dnn_model = Sequential()
dnn_model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))  # Layer 1
dnn_model.add(Dense(64, activation='relu'))  # Layer 2
dnn_model.add(Dense(10, activation='softmax'))  # Output layer

# Compile the model
dnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
dnn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the DNN model
dnn_loss, dnn_accuracy = dnn_model.evaluate(X_test, y_test)
print(f"Test accuracy (Traditional DNN): {dnn_accuracy * 100:.2f}%")

Epoch 1/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.4161 - loss: 1.9252 - val_accuracy: 0.8583 - val_loss: 0.6772
Epoch 2/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9006 - loss: 0.5276 - val_accuracy: 0.9333 - val_loss: 0.2735
Epoch 3/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9659 - loss: 0.1947 - val_accuracy: 0.9583 - val_loss: 0.1747
Epoch 4/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9774 - loss: 0.1173 - val_accuracy: 0.9694 - val_loss: 0.1378
Epoch 5/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9902 - loss: 0.0758 - val_accuracy: 0.9694 - val_loss: 0.1208
Epoch 6/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.9942 - loss: 0.0526 - val_accuracy: 0.9750 - val_loss: 0.1119
Epoch 7/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9954 - loss: 0.0455 - val_accuracy: 0.9722 - val_loss: 0.1043
Epoch 8/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9972 - loss: 0.0305 - val_accuracy: 0.9722 - val_lo